In [407]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

bastion_public_addr = 'bastion-1.fabric-testbed.net'
bastion_private_ipv4_addr = '192.168.11.226'
bastion_private_ipv6_addr = '2600:2701:5000:a902::c'

bastion_username = 'jul_0041867896'

bastion_key_filename = os.environ['HOME'] + "/.ssh/id_rsa_fabric"

ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

CM Host: cm.fabric-testbed.net Orchestrator Host: orchestrator.fabric-testbed.net
Status: Status.OK
Toplogy: NCSA: { cpu: 6/6 , core: 182/192 , ram: 1,488/1,536 G, disk: 60,390/60,600 G, unit: 3/3 }
	Components:
		NVME-P4510:  NVME P4510 { disk: 8,640/9,600 G, unit: 9/10 }
		GPU-Tesla T4:  GPU Tesla T4 { unit: 2/2 }
		GPU-RTX6000:  GPU RTX6000 { unit: 3/3 }
		SmartNIC-ConnectX-5:  SmartNIC ConnectX-5 { unit: 1/2 }
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2 }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 379/381 }
	Site Interfaces:
		NCSA_STAR: TrunkPort { bw: 100/100 Gbps}
TACC: { cpu: 10/10 , core: 320/320 , ram: 2,560/2,560 G, disk: 116,400/116,400 G, unit: 5/5 }
	Components:
		GPU-RTX6000:  GPU RTX6000 { unit: 6/6 }
		GPU-Tesla T4:  GPU Tesla T4 { unit: 4/4 }
		NVME-P4510:  NVME P4510 { disk: 15,360/15,360 G, unit: 16/16 }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 635/635 }
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2 }
		SmartNIC-ConnectX

In [491]:
slice_name = 'MySlice2'
node1_site = 'MAX'
node2_site = 'MAX'
node1_name = 'Node1'
node2_name = 'Node2'
network_service_name='p2p1'
nic1_name = 'node1-nic1'
nic2_name = 'node2-nic1'
username='centos'
image = 'default_centos_7'
image_type = 'qcow2'
cores = 32
ram = 128
disk = 50
net_service = 'L2Bridge'
use_nvme = False
use_tag_l2bridge = True

# Create Slice

In [496]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node1_name, site=node1_site)

# Set capacities
cap = Capacities()
cap.set_fields(core=cores, ram=ram, disk=disk)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Add node
n2 = t.add_node(name=node2_name, site=node2_site)

# Set properties
n2.set_properties(capacities=cap, image_type=image_type, image_ref=image)

if use_nvme:
    print("Using NVME...................")
    n1.add_component(ctype=ComponentType.NVME, model='P4510', name='c1')
    n2.add_component(ctype=ComponentType.NVME, model='P4510', name='c2')

# Smart Cards
n1.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name=nic1_name)
n2.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name=nic2_name)


if net_service == 'L2Bridge':
    # L2Bridge Service
    if use_tag_l2bridge:
        print("Using Tag...................")
        if_labels = n1.interface_list[0].get_property(pname="labels")
        if_labels.vlan = "200"
        n1.interface_list[0].set_properties(labels=if_labels)
        if_labels = n2.interface_list[0].get_property(pname="labels")
        if_labels.vlan = "200"
        n2.interface_list[0].set_properties(labels=if_labels)
    t.add_network_service(name=network_service_name, nstype=ServiceType.L2Bridge, interfaces=t.interface_list)
if net_service == 'L2PTP':
    if_labels = n1.interface_list[0].get_property(pname="labels")
    if_labels.vlan = "200"
    n1.interface_list[0].set_properties(labels=if_labels)
    if_labels = n2.interface_list[0].get_property(pname="labels")
    if_labels.vlan = "200"
    n2.interface_list[0].set_properties(labels=if_labels)
    #L2PTP Service
    t.add_network_service(name='ptp1', nstype=ServiceType.L2PTP,
                          interfaces=[n1.interface_list[0], n2.interface_list[0]])



# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                            slice_graph=slice_graph, 
                                            ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

Using Tag...................
Submitted slice creation request. Slice ID: bc7eb61d-d109-4543-bae9-0fd0426c59ba


In [497]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    


return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)

print()
print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

Waiting for slice ................... Timeout exceeded (180 sec). Slice: MySlice2 (Configuring)

Slice Name : MySlice2
ID         : bc7eb61d-d109-4543-bae9-0fd0426c59ba
State      : Configuring
Lease End  : 2021-11-20 15:23:36


# Get management IP of two nodes

In [498]:
status, slivers = slice_manager.slivers(slice_object=slices[0])

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))
else:
    print(f"Failure: {slivers}")

Response Status Status.OK
Slivers [{
    "allocated_capacities": "{\"core\": 32, \"disk\": 100, \"ram\": 128}",
    "allocated_labels": "{\"instance\": \"instance-0000068a\", \"instance_parent\": \"max-w3.fabric-testbed.net\"}",
    "capacities": "{\"core\": 32, \"disk\": 50, \"ram\": 128}",
    "capacity_hints": "{\"instance_type\": \"fabric.c32.m128.d100\"}",
    "graph_node_id": "9cf0c906-672f-4077-bc57-f8979df04b17",
    "join_state": "NoJoin",
    "lease_end": "2021-11-20 15:23:36",
    "management_ip": "63.239.135.82",
    "name": "Node2",
    "notices": "Reservation 797aa7bc-d425-45cb-ba91-d6aaff91a6ee (Slice MySlice2(bc7eb61d-d109-4543-bae9-0fd0426c59ba) Graph Id:b114acc3-f529-4676-8e56-c8553f90062a Owner:name: orchestrator guid: orchestrator-guid email: jul@clemson.edu State:StableOK ) is in state [Active,None_]\n\nLast ticket update: \nTicket events: \nLast ticket update: \nTicket events: ",
    "pending_state": "None_",
    "reservation_id": "797aa7bc-d425-45cb-ba91-d6aaff91

In [499]:
server_ip = slivers[1].management_ip
server_ip

'63.239.135.89'

In [500]:
client_ip = slivers[0].management_ip
client_ip

'63.239.135.82'

# Get the Topology

In [501]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

# Setup gridftp on server node

In [502]:
node1 = experiment_topology.nodes[node1_name]

management_ip_node1 = str(node1.get_property(pname='management_ip'))
print("Node Name        : {}".format(node1.name))
print("Management IP    : {}".format(management_ip_node1))
print()
# node2 = experiment_topology.nodes[node2_name]

# management_ip_node2 = str(node2.get_property(pname='management_ip'))
# print("Node Name        : {}".format(node2.name))
# print("Management IP    : {}".format(management_ip_node2))
# print()

Node Name        : Node1
Management IP    : 63.239.135.89



In [503]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
management_ip1 = management_ip_node1

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)

bastion=paramiko.SSHClient()
bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_key_filename)


bastion_transport = bastion.get_transport()
if validIPAddress(management_ip1) == 'IPv4':
    src_addr = (bastion_private_ipv4_addr, 22)
elif validIPAddress(management_ip1) == 'IPv6':
    src_addr = (bastion_private_ipv6_addr, 22)
else:
    print('Management IP Invalid: {}'.format(management_ip1))

dest_addr = (management_ip1, 22)
bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


client_server = paramiko.SSHClient()
client_server.load_system_host_keys()
client_server.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client_server.set_missing_host_key_policy(paramiko.AutoAddPolicy())

client_server.connect(management_ip1,username=username,pkey = key, sock=bastion_channel)

In [504]:
stdin, stdout, stderr = client_server.exec_command('sudo rpm -hUv https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Retrieving https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm
Preparing...                          ########################################
Updating / installing...
globus-toolkit-repo-6.0.9-1           ########################################
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
Loaded plugins: fastestmirror

In [505]:

stdin, stdout, stderr = client_server.exec_command('sudo yum install -y yum-plugin-priorities')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: mirrors.tripadvisor.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package yum-plugin-priorities.noarch 0:1.1.31-54.el7_8 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                    Arch        Version                 Repository
                                                                           Size
Installing:
 yum-plugin-priorities      noarch      1.1.31-54.el7_8         base       29 k

Transaction Summary
Install  1 Package

Total download size: 29 k
Installed size: 28 k
Public key for yum-plugin-priorities-1.1.31-54.el7_8.noarch.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : yum-plugin-priorities-1.1.31-54.el7_8.noarch                

In [506]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y  wget')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirrors.tripadvisor.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package wget.x86_64 0:1.14-18.el7_6.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package        Arch             Version                   Repository      Size
Installing:
 wget           x86_64           1.14-18.el7_6.1           base           547 k

Transaction Summary
Install  1 Package

Total download size: 547 k
Installed size: 2.0 M
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : wget-1.14-18.el7_6.1.x86_64                                  1/1 
  Verifying  : wget-1.14-18.el7_6.1.x86_64                                  1/1 

Installed:
  wget.x86_64 0:1.14-18.el7_6.1                                                 

Complete!




In [507]:
stdin, stdout, stderr = client_server.exec_command('sudo wget https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))


--2021-11-19 15:42:57--  https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm
Resolving download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)... 152.19.134.145, 2600:2701:4000:5211:dead:beef:fe:fed6
Connecting to download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)|152.19.134.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97432 (95K) [application/x-rpm]
Saving to: ‘udt-4.11-6.el7.x86_64.rpm’

     0K .......... .......... .......... .......... .......... 52% 1.62M 0s
    50K .......... .......... .......... .......... .....     100% 2.98M=0.04s

2021-11-19 15:42:57 (2.07 MB/s) - ‘udt-4.11-6.el7.x86_64.rpm’ saved [97432/97432]




In [508]:
stdin, stdout, stderr = client_server.exec_command('sudo rpm -Uvh *rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Preparing...                          ########################################
Updating / installing...
udt-4.11-6.el7                        ########################################




In [509]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y globus-gridftp globus-gsi globus-data-management-server')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirrors.tripadvisor.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package globus-data-management-server.x86_64 0:6.0-1.el7+gt6 will be installed
--> Processing Dependency: globus-gsi-cert-utils-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-common-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-authz for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xio-pipe-driver for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xioperf for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-usage for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Depende

In [510]:
stdin, stdout, stderr = client_server.exec_command('sudo /etc/init.d/globus-gridftp-sshftp reconfigure')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Disabling sshftp access to globus-gridftp-server: OK
Successfully created /etc/gridftp-sshftp.
Enabling sshftp access to globus-gridftp-server: OK




# Setup gridftp on client node

In [511]:
node2 = experiment_topology.nodes[node2_name]

management_ip_node2 = str(node2.get_property(pname='management_ip'))
print("Node Name        : {}".format(node2.name))
print("Management IP    : {}".format(management_ip_node2))
print()

Node Name        : Node2
Management IP    : 63.239.135.82



In [512]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
management_ip2 = management_ip_node2

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)

bastion=paramiko.SSHClient()
bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_key_filename)


bastion_transport = bastion.get_transport()
if validIPAddress(management_ip2) == 'IPv4':
    src_addr = (bastion_private_ipv4_addr, 22)
elif validIPAddress(management_ip2) == 'IPv6':
    src_addr = (bastion_private_ipv6_addr, 22)
else:
    print('Management IP Invalid: {}'.format(management_ip2))

dest_addr = (management_ip2, 22)
bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


client_client = paramiko.SSHClient()
client_client.load_system_host_keys()
client_client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

client_client.connect(management_ip2,username=username,pkey = key, sock=bastion_channel)

In [513]:
stdin, stdout, stderr = client_client.exec_command('sudo rpm -hUv https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Retrieving https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm
Preparing...                          ########################################
Updating / installing...
globus-toolkit-repo-6.0.9-1           ########################################
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
Loaded plugins: fastestmirror

In [514]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y yum-plugin-priorities')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: mirror.cogentco.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package yum-plugin-priorities.noarch 0:1.1.31-54.el7_8 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                    Arch        Version                 Repository
                                                                           Size
Installing:
 yum-plugin-priorities      noarch      1.1.31-54.el7_8         base       29 k

Transaction Summary
Install  1 Package

Total download size: 29 k
Installed size: 28 k
Public key for yum-plugin-priorities-1.1.31-54.el7_8.noarch.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : yum-plugin-priorities-1.1.31-54.el7_8.noarch                 1/1

In [515]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y  wget')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.cogentco.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package wget.x86_64 0:1.14-18.el7_6.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package        Arch             Version                   Repository      Size
Installing:
 wget           x86_64           1.14-18.el7_6.1           base           547 k

Transaction Summary
Install  1 Package

Total download size: 547 k
Installed size: 2.0 M
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : wget-1.14-18.el7_6.1.x86_64                                  1/1 
  Verifying  : wget-1.14-18.el7_6.1.x86_64                                  1/1 

Installed:
  wget.x86_64 0:1.14-18.el7_6.1                                                 

Complete!




In [516]:
stdin, stdout, stderr = client_client.exec_command('sudo wget https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))


--2021-11-19 15:47:33--  https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm
Resolving download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)... 152.19.134.145, 2600:2701:4000:5211:dead:beef:fe:fed6
Connecting to download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)|152.19.134.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97432 (95K) [application/x-rpm]
Saving to: ‘udt-4.11-6.el7.x86_64.rpm’

     0K .......... .......... .......... .......... .......... 52% 2.11M 0s
    50K .......... .......... .......... .......... .....     100% 3.82M=0.03s

2021-11-19 15:47:33 (2.68 MB/s) - ‘udt-4.11-6.el7.x86_64.rpm’ saved [97432/97432]




In [517]:
stdin, stdout, stderr = client_client.exec_command('sudo rpm -Uvh *rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Preparing...                          ########################################
Updating / installing...
udt-4.11-6.el7                        ########################################




In [518]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y globus-gridftp globus-gsi globus-data-management-server')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.cogentco.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package globus-data-management-server.x86_64 0:6.0-1.el7+gt6 will be installed
--> Processing Dependency: globus-gsi-cert-utils-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-common-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-authz for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xio-pipe-driver for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xioperf for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-usage for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency:

In [519]:
stdin, stdout, stderr = client_client.exec_command('sudo /etc/init.d/globus-gridftp-sshftp reconfigure')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Disabling sshftp access to globus-gridftp-server: OK
Successfully created /etc/gridftp-sshftp.
Enabling sshftp access to globus-gridftp-server: OK




# setup interface on server and client nodes

In [520]:
if net_service == 'L2Bridge' and not use_tag_l2bridge:
    ip_of_interface_on_n1 = "192.168.10.51"
    stdin, stdout, stderr = client_server.exec_command('ip a')
    interface_n1 = 'eth1'
    stdin, stdout, stderr = client_server.exec_command('sudo ip addr add ' + ip_of_interface_on_n1 + '/24 dev ' + interface_n1)
    stdin, stdout, stderr = client_server.exec_command('sudo ip link set dev ' + interface_n1 + ' up mtu 9000')
    print(stdout.read().decode("utf-8"))
    print(stderr.read().decode("utf-8"))
else:
    stdin, stdout, stderr = client_server.exec_command('sudo ip link add link eth1 name eth1.200 type vlan id 200')
    stdin, stdout, stderr = client_server.exec_command('sudo ip link set dev eth1 up mtu 9000')
    stdin, stdout, stderr = client_server.exec_command('sudo ip link set dev eth1.200 up mtu 9000')
    stdin, stdout, stderr = client_server.exec_command('sudo ip addr add 192.168.10.51/24 dev eth1.200')

    stdin, stdout, stderr = client_server.exec_command('ifconfig eth1.200')
    print (str(stdout.read(),'utf-8').replace('\\n','\n'))

In [521]:
if net_service == 'L2Bridge' and not use_tag_l2bridge:
    ip_of_interface_on_n2 = "192.168.10.52"
    stdin, stdout, stderr = client_client.exec_command('ip a')
    interface_n2 = 'eth1'
    stdin, stdout, stderr = client_client.exec_command('sudo ip addr add ' + ip_of_interface_on_n2 + '/24 dev ' + interface_n2)
    stdin, stdout, stderr = client_client.exec_command('sudo ip link set dev ' + interface_n2 + ' up mtu 9000')
    print(stdout.read().decode("utf-8"))
    print(stderr.read().decode("utf-8"))
else:
    stdin, stdout, stderr = client_client.exec_command('sudo ip link add link eth1 name eth1.200 type vlan id 200')
    stdin, stdout, stderr = client_client.exec_command('sudo ip link set dev eth1 up mtu 9000')
    stdin, stdout, stderr = client_client.exec_command('sudo ip link set dev eth1.200 up mtu 9000')
    stdin, stdout, stderr = client_client.exec_command('sudo ip addr add 192.168.10.52/24 dev eth1.200')

    stdin, stdout, stderr = client_client.exec_command('ifconfig eth1.200')
    print (str(stdout.read(),'utf-8').replace('\\n','\n'))

# TCP buffing size tuning on server and client node

In [522]:
stdin, stdout, stderr = client_server.exec_command( 'echo "net.core.rmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.core.wmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_rmem = 4096 87380 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_wmem = 4096 65536 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_congestion_control=htcp" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_mtu_probing=1" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.core.default_qdisc = fq" | sudo tee -a /etc/sysctl.conf')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.default_qdisc = fq

stderr 


In [523]:
stdin, stdout, stderr = client_server.exec_command('sudo sysctl -p')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.rmem_max = 2147482548
net.core.wmem_max = 2147482548
net.ipv4.tcp_rmem = 4096 87380 1073741274
net.ipv4.tcp_wmem = 4096 65536 1073741274
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq

stderr 


In [524]:

stdin, stdout, stderr = client_client.exec_command( 'echo "net.core.rmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.core.wmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_rmem = 4096 87380 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_wmem = 4096 65536 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_congestion_control=htcp" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_mtu_probing=1" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.core.default_qdisc = fq" | sudo tee -a /etc/sysctl.conf')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.default_qdisc = fq

stderr 


In [525]:
stdin, stdout, stderr = client_client.exec_command('sudo sysctl -p')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.rmem_max = 2147482548
net.core.wmem_max = 2147482548
net.ipv4.tcp_rmem = 4096 87380 1073741274
net.ipv4.tcp_wmem = 4096 65536 1073741274
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq

stderr 


# iperf test

In [526]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y epel-release')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirrors.tripadvisor.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-11 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                Arch             Version         Repository        Size
Installing:
 epel-release           noarch           7-11            extras            15 k

Transaction Summary
Install  1 Package

Total download size: 15 k
Installed size: 24 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : epel-release-7-11.noarch                                     1/1 
  Verifying  : epel-release-7-11.noarch                                     1/1 

Installed:
  epel-release.noarch 0:7-11                                                    

Complete!




In [527]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y iperf')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirrors.tripadvisor.com
 * epel: mirror.math.princeton.edu
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
155 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package iperf.x86_64 0:2.0.13-1.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package         Arch             Version                  Repository      Size
Installing:
 iperf           x86_64           2.0.13-1.el7             epel            86 k

Transaction Summary
Install  1 Package

Total download size: 86 k
Installed size: 185 k
Public key for iperf-2.0.13-1.el7.x86_64.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : iperf-2.0.13-1.el7.x86_64                          

In [528]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y epel-release')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.cogentco.com
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-11 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                Arch             Version         Repository        Size
Installing:
 epel-release           noarch           7-11            extras            15 k

Transaction Summary
Install  1 Package

Total download size: 15 k
Installed size: 24 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : epel-release-7-11.noarch                                     1/1 
  Verifying  : epel-release-7-11.noarch                                     1/1 

Installed:
  epel-release.noarch 0:7-11                                                    

Complete!




In [529]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y iperf')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.cogentco.com
 * epel: d2lzkl7pfhq30w.cloudfront.net
 * extras: nyc.mirrors.clouvider.net
 * updates: mirrors.mit.edu
155 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package iperf.x86_64 0:2.0.13-1.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package         Arch             Version                  Repository      Size
Installing:
 iperf           x86_64           2.0.13-1.el7             epel            86 k

Transaction Summary
Install  1 Package

Total download size: 86 k
Installed size: 185 k
Public key for iperf-2.0.13-1.el7.x86_64.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : iperf-2.0.13-1.el7.x86_64                          

In [530]:
stdin, stdout, stderr = client_server.exec_command('iperf -s -P32 -w 32M > /dev/null 2>&1 &')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [531]:
stdin, stdout, stderr = client_client.exec_command('iperf -c 192.168.10.51 -P 32 -w 32M')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

------------------------------------------------------------
Client connecting to 192.168.10.51, TCP port 5001
TCP window size: 64.0 MByte (WARNING: requested 32.0 MByte)
------------------------------------------------------------
[ 32] local 192.168.10.52 port 51844 connected with 192.168.10.51 port 5001
[  9] local 192.168.10.52 port 51796 connected with 192.168.10.51 port 5001
[  3] local 192.168.10.52 port 51782 connected with 192.168.10.51 port 5001
[  5] local 192.168.10.52 port 51784 connected with 192.168.10.51 port 5001
[  4] local 192.168.10.52 port 51786 connected with 192.168.10.51 port 5001
[  8] local 192.168.10.52 port 51792 connected with 192.168.10.51 port 5001
[  7] local 192.168.10.52 port 51790 connected with 192.168.10.51 port 5001
[ 10] local 192.168.10.52 port 51794 connected with 192.168.10.51 port 5001
[  6] local 192.168.10.52 port 51788 connected with 192.168.10.51 port 5001
[ 14] local 192.168.10.52 port 51798 connected with 192.168.10.51 port 5001
[ 12] lo

# gridftp performance test

In [532]:
stdin, stdout, stderr = client_server.exec_command('sudo globus-gridftp-server -control-interface 0.0.0.0  -aa  -p 8080 -anonymous-user 1040 -home-dir /tmp/ -auth-level 0 -allow-root > /dev/null 2>&1 &')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [533]:
stdin, stdout, stderr = client_server.exec_command('sudo fallocate -l 80G gentoo_root.img')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [534]:
stdin, stdout, stderr = client_client.exec_command('sudo globus-url-copy -vb -p 32 ftp://192.168.10.51:8080/home/centos/gentoo_root.img file:///dev/null')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Source: ftp://192.168.10.51:8080/home/centos/
Dest:   file:///dev/
  gentoo_root.img  ->  null

  83916488704 bytes      1482.02 MB/sec avg      1496.88 MB/sec inst




# release recources

In [489]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    for slice in slices:
        print("{}:".format(slice.slice_name))
        print("   ID         : {}".format(slice.slice_id))
        print("   State      : {}".format(slice.slice_state))
        print("   Lease End  : {}".format(slice.lease_end))
        print()
else:
    print(f"Failure: {slices}")

MySlice2:
   ID         : 67488c87-b5a3-4df5-afb7-d2d041e56d33
   State      : StableOK
   Lease End  : 2021-11-20 15:09:17



In [490]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))

Response Status Status.OK
Response received None
